In [ ]:
from pyspark.conf import SparkConf
conf = SparkConf()
conf.set('spark.executor.memory','14g')\
.set('spark.driver.memory','14g')\
.set('spark.home','/Users/deepali/Downloads/spark-3.1.2-bin-hadoop2.7')

In [ ]:
pip install 

In [ ]:
import pyspark.sql.types as typ
from pyspark.sql.types import *
from pyspark import SparkContext
from pyspark.sql import SparkSession

sc=SparkContext(conf=conf)
spark = SparkSession(sc)
# sc=SparkContext(conf=conf)
# spark = SparkSession(conf=conf)

import pandas as pd

In [ ]:
sc.sparkHome

In [ ]:
# spark=SparkContext.getOrCreate()
spark._conf.set('spark.executor.memory','32g')\
.set('spark.driver.memory','32g')\
.set('spark.driver.maxResultsSize','0')

In [ ]:
from pyspark.conf import SparkConf

In [ ]:
# conf = SparkConf(loadDefaults=False)
print(spark.conf.get("spark.home"))

In [ ]:
paths=['data2016.csv','data2017.csv','data2018.csv']
data=spark.read.csv(paths,sep=',',inferSchema='True') #merge and create dataframe

In [ ]:
# paths=['data2016.csv','data2017.csv','data2018.csv']
# data=spark.read.csv(paths,sep=',',inferSchema='True') #merge and create dataframe

#drop extra columns:
columns_to_drop = ['_c17', '_c18','_c19']
data= data.drop(*columns_to_drop)

#rename columns:
data.withColumnRenamed("_c0","ID")\
.withColumnRenamed("_c1","Name")\
.withColumnRenamed("_c2","Address")\
.withColumnRenamed("_c3","City")\
.withColumnRenamed("_c4","State")\
.withColumnRenamed("_c5","Zip")\
.withColumnRenamed("_c6","Latitude")\
.withColumnRenamed("_c7","Longitude")\
.withColumnRenamed("_c8","Summary")\
.withColumnRenamed("_c9","Serial")\
.withColumnRenamed("_c10","Number_Date")\
.withColumnRenamed("_c11","Date_Time")\
.withColumnRenamed("_c12","Score")\
.withColumnRenamed("_c13","Inspection_Type")\
.withColumnRenamed("_c14","Inspection_Time")\
.withColumnRenamed("_c15","Comment")\
.withColumnRenamed("_c16","Risk_Grade")
#data.printSchema()

#convert to integer:
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType
data = data.withColumn("_c0",col("_c0").cast(IntegerType())) \
    .withColumn("_c5",col("_c5").cast(IntegerType())) \
    .withColumn("_c6",col("_c6").cast(FloatType()))\
    .withColumn("_c7",col("_c7").cast(FloatType())) \
    .withColumn("_c9",col("_c9").cast(FloatType())) \
    .withColumn("_c12",col("_c12").cast(IntegerType())) 



#checking and dropping duplicates
print('Count of rows: {0}'.format(data.count()))
print('Count of distinct rows: {0}'.format(data.distinct().count()))
data = data.dropDuplicates()

#Drop entries with duplicates
print('Count of ids: {0}'.format(data.count()))
print('Count of distinct restaurant ids: {0}'.format(data.select([c for c in data.columns if c != '_c0']).distinct().count()))
data = data.dropDuplicates(subset=[
c for c in data.columns if c != '_c0'
])

#missing data
data1=data.fillna( { '_c6':37.7434784244497, '_c7':4122.33581867157247 ,'_c9':41.415529404676109,'_c12':4.2170298040255946E8} )
data1.show()
selected_features=['_c12','_c16',]
datatrim=data1.select(selected_features)
datatrim.show()

In [ ]:
from pyspark.sql.functions import when
data2 = datatrim.withColumn("_c16", when(datatrim._c16 == "None","1") \
                            .when(datatrim._c16 == "Low Risk","13") \
                            .when(datatrim._c16 == "Moderate Risk","14") \
                            .when(datatrim._c16 == "Routine - Unscheduled","11") \
                            .when(datatrim._c16 == "Food Borne Illness Investigation","15") \
                            .when(datatrim._c16 == "Structural Inspection","8") \
                            .when(datatrim._c16 == "Routine - Scheduled","9") \
                            .when(datatrim._c16 == "Reinspection/Followup","18") \
                            .when(datatrim._c16 == "Complaint Reinspection/Followup","17") \
                            .when(datatrim._c16 == "Non-inspection site visit","10") \
                            .when(datatrim._c16 == "Complaint","12") \
                            .when(datatrim._c16 == "High Risk","16") \
                            .when(datatrim._c16 == "New Construction","7") \
                            .when(datatrim._c16 == "New Ownership","3") \
                            .when(datatrim._c16 == "Home Environmental Assessment","2") \
                            .when(datatrim._c16 == "New Ownership Followup","4") \
                            .when(datatrim._c16 == "Special Event","5") \
                            .when(datatrim._c16 == "Administrative or Document Review","6") \
                            .otherwise(datatrim._c16))
data2= data2.withColumn("_c16",col("_c16").cast(IntegerType()))
data2.na.drop().show()

In [ ]:
from pyspark.sql.functions import lit
data3=data2.withColumn("Safety",lit(1))
data3.show()
# data_fin=data_fin.withColumn("Safety",when(data_fin["_c12"]>90, 0).otherwise(data_fin["Safety"])).show()

In [ ]:
data3
data_fin = data3.withColumn("Safety",when(data3["_c12"]> 90, 0).otherwise(data3["Safety"])).na.drop()
data_fin.show()

In [ ]:
import pyspark.mllib.feature as ft
import pyspark.mllib.linalg as ln


import pyspark.mllib.regression as reg
# features_to_keep=['_c12','_c16','_c13','Safety']
features_to_keep=['_c16','Safety']
data_final=data_fin.rdd.map(lambda row: [
[e] if type(e) == int else e for e in row
]).map(lambda row: [
item for sublist in row for item in sublist
]).map(lambda row: reg.LabeledPoint(
row[0], ln.Vectors.dense(row[1:]))
)
data_final.take(5)
# data_final

In [ ]:
data_train, data_test = data_final.randomSplit([0.8, 0.2])
data_train.take(2)
data_train

In [ ]:
data_train2=data_train.take(20)
data_train2

In [ ]:
del(data_final)
del(data_fin)
del(data3)
del(data2)
del(data)
del(data1)
del(datatrim)

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
LR_Model = LogisticRegressionWithLBFGS.train(data_train, iterations=5)

In [ ]:
 LR_results = (
           data_test.map(lambda row: row.label) \
           .zip(LR_Model \
                .predict(data_test\
                         .map(lambda row: row.features)))
       ).map(lambda row: (row[0], row[1] * 1.0))


In [ ]:
import pyspark.mllib.evaluation as ev
LR_evaluation = ev.BinaryClassificationMetrics(LR_results)
print('Area under PR: {0:.2f}'.format(LR_evaluation.areaUnderPR))
print('Area under ROC: {0:.2f}' .format(LR_evaluation.areaUnderROC))
LR_evaluation.unpersist()